In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk 
import string
import csv
from textblob import TextBlob
from tqdm import tqdm_notebook as tqdm

from google.colab import files

In [ ]:
# download dataset from GDrive
downloaded = drive.CreateFile({'id':'1KfD3JsfWXgVsGFwyRlV9G6t2Plhu_ax0'}) # use dataset's id from GDrive
downloaded.GetContentFile('dataset.csv')

# Read file as panda dataframe
dfRaw = pd.read_csv('dataset.csv')
dfRaw

,tweet,label
0,Peneliti: Presiden perlu bentuk Badan Otorita ...,0
1,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1
2,Prabowo setuju karena pemindahan ibu kota suda...,1
3,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1
4,Lah itu kan sdh mau di ringankan dg pemindahan...,1
...,...,...
2677,Semoga hal serupa juga dilakukan saat proses p...,1
2678,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1
2679,Pemindahan ibu kota berhubungan dengan CINA Ke...,0
2680,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1


In [ ]:
dfRaw[dfRaw['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfRaw[dfRaw['label'].isnull() == True]

,tweet,label


In [ ]:
# Duplicate dataframe
dfRaw_to_Stemmed = dfRaw.copy()
dfRaw_to_Pre = dfRaw.copy()
dfRaw_to_Pre_to_Stemmed = dfRaw.copy()

# dfRaw_Stemmed

In [ ]:
!pip install sastrawi

In [ ]:
# Tokenization 
dfRaw_to_Stemmed_Tokenized = dfRaw_to_Stemmed['tweet'].apply(lambda x: x.split())
dfRaw_to_Stemmed_Tokenized

0       [Peneliti:, Presiden, perlu, bentuk, Badan, Ot...
1       [Pembangunan, PLTN, di, Kalbar, mantapkan, pem...
2       [Prabowo, setuju, karena, pemindahan, ibu, kot...
3       [Pemindahan, ibu, kota, bukti, Jokowi, ingin, ...
4       [Lah, itu, kan, sdh, mau, di, ringankan, dg, p...
                              ...                        
2677    [Semoga, hal, serupa, juga, dilakukan, saat, p...
2678    [Tolak, Pemindahan, Ibu, Kota, Tolak, Kenaikan...
2679    [Pemindahan, ibu, kota, berhubungan, dengan, C...
2680    [Survei, Median:, 45,3%, Publik, Tidak, Setuju...
2681    [Masih, untung, bru, rencana..kalo, sudah, jal...
Name: tweet, Length: 2682, dtype: object

In [ ]:
from nltk.stem.porter import *
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
dfRaw_Stemmed = dfRaw_to_Stemmed_Tokenized.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
dfRaw_Stemmed

0       [teliti, presiden, perlu, bentuk, badan, otori...
1       [bangun, pltn, di, kalbar, mantap, pindah, ibu...
2       [prabowo, tuju, karena, pindah, ibu, kota, sud...
3       [pindah, ibu, kota, bukti, jokowi, ingin, ada,...
4       [lah, itu, kan, sdh, mau, di, ringan, dg, pind...
                              ...                        
2677    [moga, hal, rupa, juga, laku, saat, proses, pi...
2678    [tolak, pindah, ibu, kota, tolak, naik, iur, b...
2679    [pindah, ibu, kota, hubung, dengan, cina, rusu...
2680    [survei, median, 45 3, publik, tidak, tuju, pi...
2681    [masih, untung, bru, rencana kalo, sudah, jala...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfRaw_Stemmed = dfRaw_Stemmed.apply(lambda x: ' '.join(x))
dfRaw_Stemmed

0       teliti presiden perlu bentuk badan otorita pin...
1       bangun pltn di kalbar mantap pindah ibu kota n...
2       prabowo tuju karena pindah ibu kota sudah jadi...
3       pindah ibu kota bukti jokowi ingin ada perata ...
4        lah itu kan sdh mau di ringan dg pindah ibu kota
                              ...                        
2677    moga hal rupa juga laku saat proses pindah ibu...
2678    tolak pindah ibu kota tolak naik iur bpjs tola...
2679    pindah ibu kota hubung dengan cina rusuh di pa...
2680    survei median 45 3 publik tidak tuju pindah ib...
2681    masih untung bru rencana kalo sudah jalan kaya...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfRaw_Stemmed = pd.DataFrame(dfRaw_Stemmed, columns=['tweet', 'label'])
dfRaw_Stemmed['label'] = dfRaw.label

In [ ]:
dfRaw_Stemmed

,tweet,label
0,teliti presiden perlu bentuk badan otorita pin...,0
1,bangun pltn di kalbar mantap pindah ibu kota n...,1
2,prabowo tuju karena pindah ibu kota sudah jadi...,1
3,pindah ibu kota bukti jokowi ingin ada perata ...,1
4,lah itu kan sdh mau di ringan dg pindah ibu kota,1
...,...,...
2677,moga hal rupa juga laku saat proses pindah ibu...,1
2678,tolak pindah ibu kota tolak naik iur bpjs tola...,-1
2679,pindah ibu kota hubung dengan cina rusuh di pa...,0
2680,survei median 45 3 publik tidak tuju pindah ib...,-1


# dfPre

In [ ]:
# remove special characters, numbers, punctuations
dfRaw_to_Pre['tweet'] = dfRaw_to_Pre['tweet'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
#remove hashtag in front of words 
dfRaw_to_Pre['tweet'] = dfRaw_to_Pre['tweet'].str.replace("#([^\s]+)", " ")

In [ ]:
# converting and overwriting values in column 
dfRaw_to_Pre['tweet']= dfRaw_to_Pre['tweet'].str.lower() 

In [ ]:
# remove twitter handles (@user)
dfRaw_to_Pre['tweet'] = np.vectorize(remove_pattern)(dfRaw_to_Pre['tweet'], "@[\w]*")

# Remove URL
dfRaw_to_Pre['tweet'] = dfRaw_to_Pre['tweet'].str.replace("((www\.[^\s]+)|(https?://[^\s]+))", " ")

In [ ]:
dfRaw_to_Pre

,tweet,label
0,peneliti presiden perlu bentuk badan otorita ...,0
1,pembangunan pltn di kalbar mantapkan pemindaha...,1
2,prabowo setuju karena pemindahan ibu kota suda...,1
3,pemindahan ibu kota bukti jokowi ingin adanya ...,1
4,lah itu kan sdh mau di ringankan dg pemindahan...,1
...,...,...
2677,semoga hal serupa juga dilakukan saat proses p...,1
2678,tolak pemindahan ibu kota tolak kenaikan iuran...,-1
2679,pemindahan ibu kota berhubungan dengan cina ke...,0
2680,survei median publik tidak setuju pemin...,-1


In [ ]:
# Tokenization 
dfRaw_to_Pre_Tokenized = dfRaw_to_Pre['tweet'].apply(lambda x: x.split())
dfRaw_to_Pre_Tokenized

0       [peneliti, presiden, perlu, bentuk, badan, oto...
1       [pembangunan, pltn, di, kalbar, mantapkan, pem...
2       [prabowo, setuju, karena, pemindahan, ibu, kot...
3       [pemindahan, ibu, kota, bukti, jokowi, ingin, ...
4       [lah, itu, kan, sdh, mau, di, ringankan, dg, p...
                              ...                        
2677    [semoga, hal, serupa, juga, dilakukan, saat, p...
2678    [tolak, pemindahan, ibu, kota, tolak, kenaikan...
2679    [pemindahan, ibu, kota, berhubungan, dengan, c...
2680    [survei, median, publik, tidak, setuju, pemind...
2681    [masih, untung, bru, rencana, kalo, sudah, jal...
Name: tweet, Length: 2682, dtype: object

In [ ]:
# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [ ]:
dfRaw_to_Pre_SWRemoved_Tokenized = dfRaw_to_Pre_Tokenized.apply(lambda x: [stopword.remove(i) for i in x])

In [ ]:
dfRaw_to_Pre_SWRemoved_Tokenized

0       [peneliti, presiden, perlu, bentuk, badan, oto...
1       [pembangunan, pltn, , kalbar, mantapkan, pemin...
2       [prabowo, setuju, , pemindahan, ibu, kota, , m...
3       [pemindahan, ibu, kota, bukti, jokowi, , adany...
4       [lah, , kan, sdh, mau, , ringankan, dg, pemind...
                              ...                        
2677    [semoga, , serupa, , dilakukan, , proses, pemi...
2678    [tolak, pemindahan, ibu, kota, tolak, kenaikan...
2679    [pemindahan, ibu, kota, berhubungan, , cina, k...
2680    [survei, median, publik, , setuju, pemindahan,...
2681    [, untung, bru, rencana, kalo, , jalan, kayak,...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfRaw_Pre_SWRemoved_Tokenized_comb = dfRaw_to_Pre_SWRemoved_Tokenized.apply(lambda x: ' '.join(x))

In [ ]:
dfRaw_Pre_SWRemoved_Tokenized_comb

0       peneliti presiden perlu bentuk badan otorita p...
1       pembangunan pltn  kalbar mantapkan pemindahan ...
2       prabowo setuju  pemindahan ibu kota  menjadi u...
3       pemindahan ibu kota bukti jokowi  adanya pemer...
4       lah  kan sdh mau  ringankan dg pemindahan ibu ...
                              ...                        
2677    semoga  serupa  dilakukan  proses pemindahan i...
2678    tolak pemindahan ibu kota tolak kenaikan iuran...
2679    pemindahan ibu kota berhubungan  cina kerusuha...
2680    survei median publik  setuju pemindahan ibu ko...
2681     untung bru rencana kalo  jalan kayak bpjs gim...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfPre = pd.DataFrame(dfRaw_Pre_SWRemoved_Tokenized_comb, columns=['tweet', 'label'])
dfPre['label'] = dfRaw.label

In [ ]:
dfPre

,tweet,label
0,peneliti presiden perlu bentuk badan otorita p...,0
1,pembangunan pltn kalbar mantapkan pemindahan ...,1
2,prabowo setuju pemindahan ibu kota menjadi u...,1
3,pemindahan ibu kota bukti jokowi adanya pemer...,1
4,lah kan sdh mau ringankan dg pemindahan ibu ...,1
...,...,...
2677,semoga serupa dilakukan proses pemindahan i...,1
2678,tolak pemindahan ibu kota tolak kenaikan iuran...,-1
2679,pemindahan ibu kota berhubungan cina kerusuha...,0
2680,survei median publik setuju pemindahan ibu ko...,-1


# dfPre_Stemmed

In [ ]:
# remove special characters, numbers, punctuations
dfRaw_to_Pre_to_Stemmed['tweet'] = dfRaw_to_Pre_to_Stemmed['tweet'].str.replace("[^a-zA-Z#]", " ")

#remove hashtag in front of words 
dfRaw_to_Pre_to_Stemmed['tweet'] = dfRaw_to_Pre_to_Stemmed['tweet'].str.replace("#([^\s]+)", " ")

# converting and overwriting values in column 
dfRaw_to_Pre_to_Stemmed['tweet']= dfRaw_to_Pre_to_Stemmed['tweet'].str.lower()

# remove twitter handles (@user)
dfRaw_to_Pre_to_Stemmed['tweet'] = np.vectorize(remove_pattern)(dfRaw_to_Pre_to_Stemmed['tweet'], "@[\w]*")

# Remove URL
dfRaw_to_Pre_to_Stemmed['tweet'] = dfRaw_to_Pre_to_Stemmed['tweet'].str.replace("((www\.[^\s]+)|(https?://[^\s]+))", " ")

In [ ]:
dfRaw_to_Pre_to_Stemmed

,tweet,label
0,peneliti presiden perlu bentuk badan otorita ...,0
1,pembangunan pltn di kalbar mantapkan pemindaha...,1
2,prabowo setuju karena pemindahan ibu kota suda...,1
3,pemindahan ibu kota bukti jokowi ingin adanya ...,1
4,lah itu kan sdh mau di ringankan dg pemindahan...,1
...,...,...
2677,semoga hal serupa juga dilakukan saat proses p...,1
2678,tolak pemindahan ibu kota tolak kenaikan iuran...,-1
2679,pemindahan ibu kota berhubungan dengan cina ke...,0
2680,survei median publik tidak setuju pemin...,-1


In [ ]:
# Tokenization 
dfRaw_to_Pre_to_Stemmed_Tokenized = dfRaw_to_Pre_to_Stemmed['tweet'].apply(lambda x: x.split())
dfRaw_to_Pre_to_Stemmed_Tokenized

0       [peneliti, presiden, perlu, bentuk, badan, oto...
1       [pembangunan, pltn, di, kalbar, mantapkan, pem...
2       [prabowo, setuju, karena, pemindahan, ibu, kot...
3       [pemindahan, ibu, kota, bukti, jokowi, ingin, ...
4       [lah, itu, kan, sdh, mau, di, ringankan, dg, p...
                              ...                        
2677    [semoga, hal, serupa, juga, dilakukan, saat, p...
2678    [tolak, pemindahan, ibu, kota, tolak, kenaikan...
2679    [pemindahan, ibu, kota, berhubungan, dengan, c...
2680    [survei, median, publik, tidak, setuju, pemind...
2681    [masih, untung, bru, rencana, kalo, sudah, jal...
Name: tweet, Length: 2682, dtype: object

In [ ]:
# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [ ]:
# stopword remove
dfRaw_to_Pre_to_Stemmed_Tokenized_SWRemoved = dfRaw_to_Pre_to_Stemmed_Tokenized.apply(lambda x: [stopword.remove(i) for i in x])
dfRaw_to_Pre_to_Stemmed_Tokenized_SWRemoved

0       [peneliti, presiden, perlu, bentuk, badan, oto...
1       [pembangunan, pltn, , kalbar, mantapkan, pemin...
2       [prabowo, setuju, , pemindahan, ibu, kota, , m...
3       [pemindahan, ibu, kota, bukti, jokowi, , adany...
4       [lah, , kan, sdh, mau, , ringankan, dg, pemind...
                              ...                        
2677    [semoga, , serupa, , dilakukan, , proses, pemi...
2678    [tolak, pemindahan, ibu, kota, tolak, kenaikan...
2679    [pemindahan, ibu, kota, berhubungan, , cina, k...
2680    [survei, median, publik, , setuju, pemindahan,...
2681    [, untung, bru, rencana, kalo, , jalan, kayak,...
Name: tweet, Length: 2682, dtype: object

In [ ]:
# stemming
dfRaw_Pre_Stemmed_Tokenized_SWRemoved = dfRaw_to_Pre_to_Stemmed_Tokenized_SWRemoved.apply(lambda x: [stemmer.stem(i) for i in x])
dfRaw_Pre_Stemmed_Tokenized_SWRemoved

0       [teliti, presiden, perlu, bentuk, badan, otori...
1       [bangun, pltn, , kalbar, mantap, pindah, ibu, ...
2       [prabowo, tuju, , pindah, ibu, kota, , jadi, u...
3       [pindah, ibu, kota, bukti, jokowi, , ada, pera...
4       [lah, , kan, sdh, mau, , ringan, dg, pindah, i...
                              ...                        
2677    [moga, , rupa, , laku, , proses, pindah, ibu, ...
2678    [tolak, pindah, ibu, kota, tolak, naik, iur, b...
2679    [pindah, ibu, kota, hubung, , cina, rusuh, , p...
2680    [survei, median, publik, , tuju, pindah, ibu, ...
2681    [, untung, bru, rencana, kalo, , jalan, kayak,...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfRaw_Pre_Stemmed_Tokenized_SWRemoved_Comb = dfRaw_Pre_Stemmed_Tokenized_SWRemoved.apply(lambda x: ' '.join(x))
dfRaw_Pre_Stemmed_Tokenized_SWRemoved_Comb

0       teliti presiden perlu bentuk badan otorita pin...
1       bangun pltn  kalbar mantap pindah ibu kota neg...
2       prabowo tuju  pindah ibu kota  jadi usul renca...
3         pindah ibu kota bukti jokowi  ada perata bangun
4             lah  kan sdh mau  ringan dg pindah ibu kota
                              ...                        
2677    moga  rupa  laku  proses pindah ibu kota  pros...
2678    tolak pindah ibu kota tolak naik iur bpjs tola...
2679    pindah ibu kota hubung  cina rusuh  papua hubu...
2680      survei median publik  tuju pindah ibu kota news
2681     untung bru rencana kalo  jalan kayak bpjs gim...
Name: tweet, Length: 2682, dtype: object

In [ ]:
dfPre_Stemmed = pd.DataFrame(dfRaw_Pre_Stemmed_Tokenized_SWRemoved_Comb, columns=['tweet', 'label'])
dfPre_Stemmed['label'] = dfRaw.label

In [ ]:
dfPre_Stemmed

,tweet,label
0,teliti presiden perlu bentuk badan otorita pin...,0
1,bangun pltn kalbar mantap pindah ibu kota neg...,1
2,prabowo tuju pindah ibu kota jadi usul renca...,1
3,pindah ibu kota bukti jokowi ada perata bangun,1
4,lah kan sdh mau ringan dg pindah ibu kota,1
...,...,...
2677,moga rupa laku proses pindah ibu kota pros...,1
2678,tolak pindah ibu kota tolak naik iur bpjs tola...,-1
2679,pindah ibu kota hubung cina rusuh papua hubu...,0
2680,survei median publik tuju pindah ibu kota news,-1


In [ ]:
dfPre[dfPre['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfPre_Stemmed[dfPre_Stemmed['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfPre.loc[885, :]

tweet     
label    0
Name: 885, dtype: object

# Export

In [ ]:
dfRaw.to_csv('dfRaw.csv', index = False)
dfRaw_Stemmed.to_csv('dfRaw_Stemmed.csv', index = False)
dfPre.to_csv('dfPre.csv', index = False)
dfPre_Stemmed.to_csv('dfPre_Stemmed.csv', index = False)

files.download("dfRaw.csv")
files.download("dfRaw_Stemmed.csv")
files.download("dfPre.csv")
files.download("dfPre_Stemmed.csv")

# Check for NaN Data

In [ ]:
# download dataset from GDrive
downloaded1 = drive.CreateFile({'id':'1CPRImIDzHRSaaTj8pIlG-RLOKeFaNzHN'})
downloaded2 = drive.CreateFile({'id':'1-2q-v-UuZR9S8AZcqkpOdPo0cgaXnJ9i'})
downloaded3 = drive.CreateFile({'id':'1re1MrFMxEqW-CIBsMb-mgWR92njIYgFb'})
downloaded4 = drive.CreateFile({'id':'1t4jRcEyFyU3Xa7DLb3Rv9ZNfDiNLGdjU'})

downloaded1.GetContentFile('dataset1.csv')
downloaded2.GetContentFile('dataset2.csv')
downloaded3.GetContentFile('dataset3.csv')
downloaded4.GetContentFile('dataset4.csv')

# Read file as panda dataframe
dfRaw = pd.read_csv('dataset1.csv')
dfRaw_Stemmed = pd.read_csv('dataset2.csv')
dfPre = pd.read_csv('dataset3.csv')
dfPre_Stemmed = pd.read_csv('dataset4.csv')

In [ ]:
print("Banyak dfRaw         : ", len(dfRaw))
print("Banyak dfRaw_Stemmed : ", len(dfRaw_Stemmed))
print("Banyak dfPre         : ", len(dfPre))
print("Banyak dfPre_Stemmed : ", len(dfPre_Stemmed))

Banyak dfRaw         :  2682
Banyak dfRaw_Stemmed :  2682
Banyak dfPre         :  2682
Banyak dfPre_Stemmed :  2682


In [ ]:
dfRaw[dfRaw['label'].isnull() == True]

,tweet,label


In [ ]:
dfRaw[dfRaw['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfRaw_Stemmed[dfRaw_Stemmed['label'].isnull() == True]

,tweet,label


In [ ]:
dfRaw_Stemmed[dfRaw_Stemmed['tweet'].isnull() == True]

,tweet,label


In [ ]:
dfPre[dfPre['label'].isnull() == True]

,tweet,label


In [ ]:
dfPre[dfPre['tweet'].isnull() == True]

,tweet,label
885,NaN,0
2438,NaN,1


In [ ]:
dfPre_Stemmed[dfPre_Stemmed['label'].isnull() == True]

,tweet,label


In [ ]:
dfPre_Stemmed[dfPre_Stemmed['tweet'].isnull() == True]

,tweet,label
885,NaN,0
2438,NaN,1


In [ ]:
dfPre_Stemmed = dfPre_Stemmed.drop([885, 2438])

In [ ]:
dfPre = dfPre.drop([885, 2438])

In [ ]:
dfRaw = dfRaw.drop([885, 2438])

In [ ]:
dfRaw_Stemmed = dfRaw_Stemmed.drop([885, 2438])

In [ ]:
print("Banyak dfRaw         : ", len(dfRaw))
print("Banyak dfRaw_Stemmed : ", len(dfRaw_Stemmed))
print("Banyak dfPre         : ", len(dfPre))
print("Banyak dfPre_Stemmed : ", len(dfPre_Stemmed))

Banyak dfRaw         :  2680
Banyak dfRaw_Stemmed :  2680
Banyak dfPre         :  2680
Banyak dfPre_Stemmed :  2680


In [ ]:
dfRaw.to_csv('dfRaw.csv', index = False)
dfRaw_Stemmed.to_csv('dfRaw_Stemmed.csv', index = False)
dfPre.to_csv('dfPre.csv', index = False)
dfPre_Stemmed.to_csv('dfPre_Stemmed.csv', index = False)

files.download("dfRaw.csv")
files.download("dfRaw_Stemmed.csv")
files.download("dfPre.csv")
files.download("dfPre_Stemmed.csv")